## Endpoints DCAT

Recommandationns pour le POC Qualité : 
- Pas d'impératif de travailler sur du temps, possible de s'appuyer un dump des graphes ou de la base PostgreSQL

Liens utiles : 
- Endpoint DCAT POC CKAN, exemple : https://preprod.data.developpement-durable.gouv.fr/dcat/catalog/jsonld?page=1
- Endpoint DCAT data.gouv, exemple : https://www.data.gouv.fr/catalog.jsonld?page=1
- Endpoint DCAT GeoNetworks, exemple : https://catalogue.datara.gouv.fr/geonetwork/srv/fre/rdf.search? (à vérifier)
- [Documentation API data.gouv](https://doc.data.gouv.fr/api/reference/#/), en particulier */site/catalog.{format}*
- [Documentation data.gouv](https://doc.data.gouv.fr/api/telecharger-un-catalogue-de-donnees/) spécifiquement sur le téléchargement et la consulation du catalogue en RDF.
- [Documention GeoNetwork](https://geonetwork-opensource.org/manuals/4.0.x/en/api/rdf-dcat.html) endpoint DCAT
- [Librairie Python RDFLib](https://rdflib.readthedocs.io/en/stable/intro_to_graphs.html) : navigation dans un graph RDF.


Tests :
- POC CKAN : comparer le temps de requête d'un graphe entre différentes pages.
- Extraire des métadonnées et les convertir en DataFrame.
- Comparer les expositions DCAT data.gouv, GeoNetworks, POC CKAN

### DCAT POC CKAN

#### Temps de réponse de l'API

In [16]:
from rdflib import Graph, URIRef
import time
import pandas as pd

pages = [1, 50, 100, 200]

for page in pages:
    try:
        start_time = time.time()
        Graph().parse(f"https://preprod.data.developpement-durable.gouv.fr/dcat/catalog/jsonld?page={page}")
        print(f"Page {page}:", time.time()-start_time)
    except Exception as exception:
        print(f"Page {page}: ", exception)

Page 1: 31.023728370666504
Page 50:  HTTP Error 504: Gateway Time-out
Page 100: 39.53203773498535
Page 200: 37.09558844566345


#### Récupération de l'ID, du titre, de la description et de la licence des datasets

In [5]:
graph = Graph().parse(f"https://preprod.data.developpement-durable.gouv.fr/dcat/catalog/jsonld?page=1")

In [74]:
data = []
for subject, predicate, object in graph.triples((None, URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"), URIRef("http://www.w3.org/ns/dcat#Dataset"))):
    data.append([subject,
                graph.value(subject=subject, predicate=URIRef("http://purl.org/dc/terms/title")),
                graph.value(subject=subject, predicate=URIRef("http://purl.org/dc/terms/description")),
                graph.value(subject=subject, predicate=URIRef("http://purl.org/dc/terms/modified")), 
                ])
                    
datasets = pd.DataFrame(data, columns= ["Object",
                                        "Title",
                                        "Description",
                                        "Modified"])